In [3]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import plotly.express as px
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.metrics import silhouette_samples
!pip install prince
import prince

# Otras tecnicas de agrupamiento


## Minibatch K-means
En esta técnica se subdividen los individuos en grupos más pequeños, y en cada uno se realiza el algoritmo kmeans. Luego se calcula el centroide promedio (por lo general) y sobre este se calculan los grupos.
 La documentacion está [acá](https://scikit-learn.org/stable/modules/generated/sklearn.cluster.MiniBatchKMeans.html#sklearn.cluster.MiniBatchKMeans)
 

In [4]:
url="https://raw.githubusercontent.com/Cruzalirio/Ucentral/master/Bases/ICFES/PruebaSaber1.csv"
datos1=pd.read_csv(url, sep=";", encoding="latin1")
for i in range(11):
 j=i+2
 x=["https://raw.githubusercontent.com/Cruzalirio/Ucentral/master/Bases/ICFES/PruebaSaber",j, ".csv"]
 x[1]=str(x[1])
 url="".join(x)
 print(url)
 datosd=pd.read_csv(url, sep=";", encoding="latin1")
 datos1=pd.concat([datos1, datosd])

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning:

Columns (9,57,69) have mixed types.Specify dtype option on import or set low_memory=False.



https://raw.githubusercontent.com/Cruzalirio/Ucentral/master/Bases/ICFES/PruebaSaber2.csv


/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning:

Columns (9,17,57,69) have mixed types.Specify dtype option on import or set low_memory=False.



https://raw.githubusercontent.com/Cruzalirio/Ucentral/master/Bases/ICFES/PruebaSaber3.csv
https://raw.githubusercontent.com/Cruzalirio/Ucentral/master/Bases/ICFES/PruebaSaber4.csv
https://raw.githubusercontent.com/Cruzalirio/Ucentral/master/Bases/ICFES/PruebaSaber5.csv
https://raw.githubusercontent.com/Cruzalirio/Ucentral/master/Bases/ICFES/PruebaSaber6.csv
https://raw.githubusercontent.com/Cruzalirio/Ucentral/master/Bases/ICFES/PruebaSaber7.csv
https://raw.githubusercontent.com/Cruzalirio/Ucentral/master/Bases/ICFES/PruebaSaber8.csv
https://raw.githubusercontent.com/Cruzalirio/Ucentral/master/Bases/ICFES/PruebaSaber9.csv
https://raw.githubusercontent.com/Cruzalirio/Ucentral/master/Bases/ICFES/PruebaSaber10.csv
https://raw.githubusercontent.com/Cruzalirio/Ucentral/master/Bases/ICFES/PruebaSaber11.csv
https://raw.githubusercontent.com/Cruzalirio/Ucentral/master/Bases/ICFES/PruebaSaber12.csv


In [5]:
datos=datos1.dropna()
cuanti=datos.select_dtypes(exclude=["object"])
cuali=datos.select_dtypes(include=["object"])
# Estandarizar y ACP
escala=StandardScaler(with_mean=True, with_std=True)
escala.fit(cuanti)
datosestan=escala.transform(cuanti)
## Realizo el ACP
pca=PCA(0.9)
pca.fit(datosestan)  ## Ajusto el PCA (valores, vectpores, varianza)
nuevosACP=pca.transform(datosestan)
pca.explained_variance_ratio_
nuevosACP.shape

(106738, 5)

In [11]:
%%time
within= [] ### Grafico de codo con Kmeans
for k in range(1,10):
    kmeanModel = KMeans(n_clusters=k).fit(nuevosACP)
    within.append(kmeanModel.inertia_)
fig=px.line(x=list(range(1,10)), y=within)
fig.show()

CPU times: user 41.4 s, sys: 10 s, total: 51.5 s
Wall time: 39.9 s


In [12]:
%%time
from sklearn.cluster import MiniBatchKMeans
within= []
for k in range(1,10):
    kmeanModel = MiniBatchKMeans(n_clusters=k).fit(nuevosACP)
    within.append(kmeanModel.inertia_)
fig=px.line(x=list(range(1,10)), y=within)
fig.show()

CPU times: user 6.22 s, sys: 2.27 ms, total: 6.22 s
Wall time: 6.23 s


In [31]:
#### Cuantos grupos harian (2)
kmedias = KMeans(n_clusters=2).fit(nuevosACP)  ### Consume 
MBkmedias= MiniBatchKMeans(n_clusters=2).fit(nuevosACP)

In [32]:
pd.crosstab(kmedias.labels_,MBkmedias.labels_,margins=True)

col_0,0,1,All
row_0,,,
0,52427,455,52882
1,0,53856,53856
All,52427,54311,106738


In [8]:
### Ejecutenlo en sus casitas
base=nuevosACP
within= {}
results = {}
for k in range(2,10):
    kmeanModel = MiniBatchKMeans(n_clusters=k).fit(base)
    within[k]=kmeanModel.inertia_
    predictions = kmeanModel.predict(base)
    results[k] = silhouette_score(base, predictions)
    print(k)

NameError: ignored

In [ ]:
results

{2: 0.32246336100989653,
 3: 0.2365156632486427,
 4: 0.2359506552427224,
 5: 0.19840793578904772,
 6: 0.18864158892249885,
 7: 0.18840851925705754,
 8: 0.1823046900329551,
 9: 0.18465987158132843}

# K-mediods

Vamos a instalar el paquete [pyclustering](https://pypi.org/project/pyclustering/) y toda la documentación está acá [PYC](https://pyclustering.github.io/docs/0.9.0/html/index.html)


In [33]:
!pip install pyclustering


     |████████████████████████████████| 2.7MB 3.2MB/s 
  Created wheel for pyclustering: filename=pyclustering-0.10.0.1-cp36-none-any.whl size=2615590 sha256=119d40761e722c0ce24006f364dd888d8d63726b6020b0a03a5018a6375e97cc
  Stored in directory: /root/.cache/pip/wheels/7c/eb/8d/ce15daecf5a50483a8fbaf1d1e3cb2d7bbaeb0ad709af32419
Successfully built pyclustering


In [34]:
url="https://raw.githubusercontent.com/Cruzalirio/Ucentral/master/Bases/Delitos_Colombia.csv"
cuanti=pd.read_csv(url, sep=";",index_col=0)
cuanti["DSP"]=cuanti.Delitos_Sexuales/cuanti.Poblacion
cuanti["HP"]=cuanti.Homicidios/cuanti.Poblacion
cuanti["IP"]=cuanti.Intrafamiliar/cuanti.Poblacion
cuanti["TP"]=cuanti.Transito/cuanti.Poblacion
cuanti["AP"]=cuanti.Asalto/cuanti.Poblacion
DC=cuanti[["DSP", "HP", "IP", "TP", "AP"]]
## Estandarizar 
escala=StandardScaler(with_mean=True, with_std=True)
escala.fit(DC)
datosestan=escala.transform(DC)
## Realizo el ACP
pca=PCA(0.9)
pca.fit(datosestan)  ## Ajusto el PCA (valores, vectpores, varianza)
nuevosACP=pca.transform(datosestan)
pca.explained_variance_ratio_

array([0.52799782, 0.21619624, 0.13499712, 0.06263963])

In [37]:
from pyclustering.cluster.kmedoids import kmedoids ## menos sensible a datos atipicos
kmed=kmedoids(nuevosACP,[1,2]) ### la longitud del segundo argumento es el numero de grupos
kmed.process()
a=kmed.get_clusters()
a  ## Problema, como le asigno ese agrupamiento a la base original

[[0,
  1,
  2,
  3,
  4,
  5,
  6,
  7,
  8,
  9,
  10,
  11,
  12,
  13,
  14,
  16,
  17,
  19,
  20,
  21,
  22,
  23,
  26,
  30,
  31,
  32],
 [15, 18, 24, 25, 27, 28, 29]]

In [38]:
DC
## Adicionar esos labels a la base original

,DSP,HP,IP,TP,AP
Departamento,,,,,
Antioquia,0.000323,0.000056,0.001226,0.000759,0.001778
Atlantico,0.000409,0.000033,0.001437,0.000757,0.000023
Bogota_D.C.,0.000515,0.000179,0.002422,0.000089,0.000333
Bolivar,0.000435,0.000013,0.000960,0.000425,0.001755
Boyaca,0.000403,0.000074,0.002112,0.000910,0.003186
Caldas,0.000446,0.000065,0.000981,0.001256,0.002257
Caqueta,0.000445,0.000036,0.000905,0.000244,0.001318
Cauca,0.000297,0.000044,0.001042,0.000608,0.001679
Cesar,0.000526,0.000041,0.001316,0.000790,0.002587


# CLARA

In [43]:
%%time
from pyclustering.cluster.clarans import clarans
clara=clarans(nuevosACP,number_clusters=2, numlocal=3, maxneighbor=10)
clara.process()
a=clara.get_clusters()
a

CPU times: user 8.57 s, sys: 1.89 ms, total: 8.57 s
Wall time: 8.58 s


# DBSCAN 
Cambia el paradigma de distancia

In [52]:
from sklearn.cluster import DBSCAN
dbscan=DBSCAN(eps=2).fit(nuevosACP)
dbscan.labels_

array([ 0,  0, -1,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0, -1,  0,
        0,  0,  0,  0,  0,  0,  0, -1, -1,  0, -1, -1, -1,  0,  0,  0])

In [54]:
dbscan=DBSCAN(eps=1.5).fit(nuevosACP)

features = DC.columns

components = pca.fit_transform(datosestan)

loadings = pca.components_.T * np.sqrt(pca.explained_variance_)

fig = px.scatter(components, x=0, y=1, color=dbscan.labels_.astype(str))

for i, feature in enumerate(features):
    fig.add_shape(
        type='line',
        x0=0, y0=0,
        x1=loadings[i, 0],
        y1=loadings[i, 1]
    )
    fig.add_annotation(
        x=loadings[i, 0],
        y=loadings[i, 1],
        ax=0, ay=0,
        xanchor="center",
        yanchor="bottom",
        text=feature,
    )
fig.show()

# Fuzzy Clustering

In [ ]:
from pyclustering.samples.definitions import FAMOUS_SAMPLES
from pyclustering.cluster import cluster_visualizer
from pyclustering.cluster.center_initializer import kmeans_plusplus_initializer
from pyclustering.cluster.fcm import fcm
from pyclustering.utils import read_sample
# load list of points for cluster analysis
sample = read_sample(FAMOUS_SAMPLES.SAMPLE_OLD_FAITHFUL)
# initialize
initial_centers = kmeans_plusplus_initializer(sample, 2, kmeans_plusplus_initializer.FARTHEST_CENTER_CANDIDATE).initialize()
# create instance of Fuzzy C-Means algorithm
fcm_instance = fcm(sample, initial_centers)
# run cluster analysis and obtain results
fcm_instance.process()
clusters = fcm_instance.get_clusters()
centers = fcm_instance.get_centers()
# visualize clustering results
visualizer = cluster_visualizer()
visualizer.append_clusters(clusters, sample)
visualizer.append_cluster(centers, marker='*', markersize=10)
visualizer.show()

ModuleNotFoundError: ignored